<a href="https://colab.research.google.com/github/bradygm/PID-controller-simulator/blob/main/PID_controller_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RoboLaunch Workshop Series: **Intro to PID Control**

In this notebook you will

If you didn't see the presentation linked to this Colab, you can watch it here: [**How to Control a Robot: Intuition for Feedback Control and PID**]() 

### Quick Google Colab Intro

A notebook is a list of cells. Cells contain either explanatory text or executable code and its output. Click a cell to select it.

Below is a **code cell** that prints "Hello World!". Click in the cell to select it and execute the contents in one of the following ways:

* Click the **Play icon** in the left gutter of the cell;
* Type **Cmd/Ctrl+Enter** to run the cell in place;
* Type **Shift+Enter** to run the cell and move focus to the next cell (adding one if none exists); 

In [ ]:
# This is a comment
print('Hello World!')

Hello World!


Did you see the code cell print "Hello World!"? Great!

# Example 1: Single link robot arm

Our first example is going to be a single link robot arm. This is essentially a rod with one pivot at the end. At the pivot point there is a motor we can control in order to get the arm to rotate in either direction. This rotational force is called **torque**.

The angle of the arm is defined as in the image below:

<img src="https://raw.githubusercontent.com/bradygm/PID-controller-simulator/fc1fc67be86c934b0a07869eaddb00a79ff07488/media/armAngle.png" alt="drawing" width="200"/>






## Imports, helper functions, and classes

We will start by running some code that will set everything up for us. You can run all of these cells by just clicking on the **Play icon** below next to the text that reads "*7 cells hidden*". You can also feel free to open the cells later to look through the code and modify. You can collapse the celss again by clicking on the arrow next to the title of this text box. 

##### Imports

In [65]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import math
from IPython.display import HTML, Javascript
from scipy import interpolate
from ipywidgets import interact, interactive, fixed, interact_manual, Video
import ipywidgets as widgets
import time
from scipy import signal
import urllib.request
import requests, warnings

##### CustomInput Class, interpolation function, and display function

In [66]:
class CustomInput:

    def __init__(self, min_val, max_val):
        self.a = widgets.FloatSlider(description='Input',min=min_val, max=max_val)
        self.values = np.array([[]])

        self.start_time = -1.0
        self.first_time = True

    def f(self, a):
        if self.first_time:
            self.first_time = False
            plt.plot(self.values[:,0], self.values[:,1])
            plt.xlabel("Seconds")
            plt.ylabel("Input")
            return

        if self.start_time == -1.0:
            self.start_time = time.time()

        self.values = np.append(self.values, [[time.time()-self.start_time, a]], axis=0)
        plt.plot(self.values[:,0], self.values[:,1], drawstyle="steps-pre")
        plt.xlabel("Seconds")
        plt.ylabel("Input")

    def record(self):
        self.values = np.empty((0,2), float)
        self.start_time = -1.0
        self.first_time = True

        out = widgets.interactive_output(self.f, {'a': self.a})

        # vid_out = widgets.Output(layout={'border': '1px solid black'})
        # with vid_out:
        #     display(Video.from_file("animation.mp4", play=True))
        # widgets.HBox([widgets.VBox([a]), out, vid_out])
        return widgets.HBox([widgets.VBox([self.a]), out])

In [67]:
def interp_values(values, end_time, dt):
    # values_t = np.array(values)
    f = interpolate.interp1d(values[:,0], values[:,1], kind='previous', fill_value="extrapolate")
    time_list = np.arange(0, end_time, dt)
    return f(time_list)

In [68]:
# def submit_response(Name="AnonUser", Error=99999999, P=0, D=0, verbose=False):
def submit_response(Name="AnonUser", verbose=False):
    end_time = 10 
    period = 10 
    start_angle = 0.0 
    dt = 0.05 
    time_list = np.arange(0, end_time, dt)
    list_len = len(time_list)
    total_error = 0

    # Initialize the RobotArm and save as my_robot_arm
    my_robot_arm = RobotArm(start_angle, dt)

    goal_ang_interp = signal.square(2 * np.pi * time_list/period)

    for i in range(list_len):
        total_error += abs(goal_ang_interp[i]-my_robot_arm.get_ang_position())*dt
        torque = pd_controller(P, D, 
                            goal_ang_interp[i], 
                            my_robot_arm.get_ang_position(),
                            my_robot_arm.get_ang_velocity())
        my_robot_arm.step_forward(torque)


    url = 'https://docs.google.com/forms/d/e/1FAIpQLSdRzW6JKXLfBW9q4KQCL6ijfs_NoAvpuXqCU-ik0H5XROilNA/viewform'
    submit_url = url.replace('/viewform', '/formResponse')
    form_data = {'draftResponse':[],
                'pageHistory':0}
    form_data['entry.1119491454'] = total_error
    form_data['entry.1429935538'] = Name
    form_data['entry.1469728088'] = P
    form_data['entry.1372586630'] = D
    print("Total Error: " + str(total_error))
    if verbose:
        print(form_data)
    user_agent = {'Referer':url,
                  'User-Agent': "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.52 Safari/537.36"}
    return requests.post(submit_url, data=form_data, headers=user_agent)

In [69]:
#############################################
# Display the results
def display_results(goal_ang_interp):
    start = time.time()
    # arm sim
    fig = plt.figure(figsize=(5, 4))
    ax = fig.add_subplot(autoscale_on=False, xlim=(-my_robot_arm.length*1.2, my_robot_arm.length*1.2), ylim=(-my_robot_arm.length*1.2, my_robot_arm.length*1.2))
    ax.set_aspect('equal')

    line, = ax.plot([], [], 'o-', lw=6)
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)


    def animate_arm(i):
        thisx = [0, my_robot_arm.length*np.sin(ang_hist[i])]
        thisy = [0, my_robot_arm.length*np.cos(ang_hist[i])]

        line.set_data(thisx, thisy)
        time_text.set_text(time_template % (i*my_robot_arm.timestep))
        return line, time_text

    ani_arm_vid = animation.FuncAnimation(
        fig, animate_arm, len(ang_hist), interval=my_robot_arm.timestep*1000, blit=True)
    # gitwriter = animation.ImageMagickWriter()
    # writergif = animation.PillowWriter(fps=50) 
    # ani_arm_vid.save('animation3.gif', writer=writergif)
    plt.close()

    # arm postion vs goal position
    fig = plt.figure(figsize=(6, 4))
    ax = fig.add_subplot(autoscale_on=False, xlim=(0, end_time), ylim=(-np.pi, np.pi))
    plt.title("Arm Angle and Goal Angle")
    plt.xlabel("Seconds")
    plt.ylabel("Angle (Radians)")
    ax.grid()

    goal_list_size = len(goal_ang_interp)

    line2, = ax.plot([], [], '--', lw=2)
    line2.set_label('Arm Angle')
    line3, = ax.plot(time_list[0], ang_hist[0], 'o', lw=2)
    if goal_list_size != 0:
        line4, = ax.plot(time_list[0:goal_list_size-1], goal_ang_interp[0:goal_list_size-1], '--', lw=2)
        line4.set_label('Arm Goal Angle')
    time_template2 = 'time = %.1fs'
    time_text2 = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    # ax.legend()


    def animate_arm2(i):
        line2.set_data(time_list[0:i], ang_hist[0:i])
        line3.set_data(time_list[i], ang_hist[i])
        time_text2.set_text(time_template2 % (i*my_robot_arm.timestep))

        return line2, time_text2

    ani_arm_vid2 = animation.FuncAnimation(
        fig, animate_arm2, len(ang_hist), interval=my_robot_arm.timestep*1000, blit=True)
    plt.close()

    #################################
    # display each animation next to each other
    window1 = widgets.Output(layout={'border': '1px solid black'})
    window2 = widgets.Output(layout={'border': '1px solid black'})
    window3 = widgets.Output(layout={'border': '1px solid black'})
    
    # display arm sim
    with window1:
        if animate_sim:
            display(HTML(ani_arm_vid.to_html5_video()))
    # display angle info
    with window2:
        if animate_plots:
            display(HTML(ani_arm_vid2.to_html5_video()))
        else:
            fig = plt.figure(figsize=(6, 4))
            ax = fig.add_subplot(autoscale_on=False, xlim=(0, end_time), ylim=(-np.pi, np.pi))
            ax.grid()
            plt.title("Arm Angle and Goal Angle")
            plt.xlabel("Seconds")
            plt.ylabel("Angle (Radians)")
            if goal_list_size != 0:
                line2, = ax.plot(time_list[0:list_len-1], goal_ang_interp[0:list_len-1], '--', lw=2)
                line2.set_label('Arm Goal Angle')
            line3, = ax.plot(time_list[0:list_len-1], ang_hist[0:list_len-1], '--', lw=2)
            line3.set_label('Arm Angle')
            ax.legend()
            plt.show()
    # display input torque info
    with window3:
            fig = plt.figure(figsize=(6, 4))
            ax = fig.add_subplot(autoscale_on=False, xlim=(0, end_time), ylim=(np.min(torques_interp)*1.2, np.max(torques_interp)*1.2))
            ax.grid()
            plt.title("Torque")
            plt.xlabel("Seconds")
            plt.ylabel("Input")
            # if goal_list_size != 0:
            #     line4, = ax.plot(time_list[0:list_len-1], goal_ang_interp[0:list_len-1], '--', lw=2)
            line5, = ax.plot(time_list[0:list_len-1], torques_interp[0:list_len-1], 'g-', lw=2)
            line5.set_label('Torque')
            ax.legend()
            plt.show()

        
    print(str(time.time()-start) + " sec")

    # FFwriter = animation.FFMpegWriter()
    # ani_arm_vid.save('animation.mp4', writer = FFwriter)
    # gitwriter = animation.ImageMagickWriter()
    # ani_arm_vid.save('animation.gif', writer=gitwriter)
    # ani_arm_vid2.save('animation2.mp4', writer = FFwriter)
    # widgets.HBox([widgets.VBox([a]), out, vid_out])
    return widgets.HBox([window1, window2, window3])
    
    

## Define the Robot Arm

Run the cell below to create your robot arm. You can adjust the parameters of the arm with the toggles on the right, but probably just leave them as they are at first.
<!--
Any cell you see with these adjustable parameters can show or hide its associated code. You can show the code by clicking "Show code" at the bottom of the cell or just by double clicking on the cell itself. In order to hide the code, just double click on parameter box or right click on the cell and select **Form → Show/hide code**.
--> 

In [39]:
# @markdown # Robot Arm Parameters { display-mode: "form" }
# @markdown You can adjust parameters using the sliders and form below.
# @markdown Remember you need to **rerun this
# @markdown cell after you pick new parameters**.

# @markdown ---
class RobotArm:

    def __init__(self, start_angle, timestep):
        # @markdown Mass of the robot arm:
        self.mass = 1 #@param{type:"slider", min:0.1, max:10, step:0.1}
        # @markdown Length of the robot arm:
        self.length =  1#@param{type:"slider", min:0.1, max:10, step:0.1}
        # @markdown Damping from friction
        self.damping = 0.1 #@param
        # @markdown Toggle gravity on or off
        self.gravity_on = False #@param{type:"boolean"}
        
        self.state = np.array([start_angle, 0.0])
        self.gravity = 9.8
        self.timestep = timestep

    def state_derivative(self, state, torque):
        ang_position = state[0]
        ang_velocity = state[1]
        if self.gravity_on:
            accelaration = (3/self.mass/self.length**2)*(torque - self.damping*ang_velocity + self.mass*self.gravity*self.length/2*np.sin(ang_position))
        else:
            accelaration = 3*(torque-ang_velocity*self.damping)/(self.mass*self.length**2)

        return np.array([ang_velocity, accelaration])
        
    def step_forward(self, torque): 
        # Using Runge-Kutta algorithm
        k1 = self.state_derivative(self.state, torque)
        k2 = self.state_derivative(self.state + self.timestep/2*k1, torque)
        k3 = self.state_derivative(self.state + self.timestep/2*k2, torque)
        k4 = self.state_derivative(self.state + self.timestep*k3, torque)
        self.state = self.state + self.timestep/6 * (k1 + 2*k2 + 2*k3 + k4)

    def angle_wrap(self, ang):
        return (ang + np.pi) % (2 * np.pi) - np.pi

    def get_ang_position(self):
        return self.state[0]
    
    def get_ang_velocity(self):
        return self.state[1]

## Let's move the arm around!

Now you get to move the arm! You will do this by varying the torque at the pivot of the arm, just like as if there was a motor there. 


First we will create a sequence of torques. Run the cell below and you should see a slider. Drag the slider around and it will save your input in realtime and display it to the graph on the right. Positive torque will add force in the clockwise direction and negative torque in the counter-clockwise direction.

We will be using the **first 10 seconds** of input in the following steps. 


In [40]:
# Creates the CustomInput object for our input torque sequence
custom_torques = CustomInput(-1,1)
# Starts recording a new torque sequence
custom_torques.record()

### Simulate the result

Now we will simulate the effects of the input you recorded.

Keep the default parameters for now, but feel free to modify later. 

In [41]:
#@markdown How long you want the simulation to run for
end_time = 10.0 #@param{type:"slider", min:5.0, max:30, step:1} 
# The starting angle of the arm
start_angle = 0.0 
dt = 0.05 # the length of the time step
#@markdown Toggle animation of figures on or off. Turning all off will result in quick results.
animate_plots = False #@param {type:"boolean"}
animate_sim = True #@param{type:"boolean"}
time_list = np.arange(0, end_time, dt)
list_len = len(time_list)
ang_hist = []
goal_ang_interp = []

# Initialize the RobotArm and save as my_robot_arm
my_robot_arm = RobotArm(start_angle, dt)

# Takes our custom torque input and interpolates it for all time steps
torques_interp = interp_values(custom_torques.values, end_time, dt)

# Loop over the time steps
for i in range(list_len):
    my_robot_arm.step_forward(torques_interp[i])
    ang_hist.append(my_robot_arm.get_ang_position())

display_results(goal_ang_interp)

10.30936074256897 sec


You can go back and try different torque sequences. Run the torque input cell again to put in new sequence and then run the simulate again to see the results. 

### Follow a goal path

Now we will see if we can get the arm to do something a bit challenging. See if you can get the arm to follow as closely as follows in the example below:

<img src="https://raw.githubusercontent.com/bradygm/PID-controller-simulator/main/media/animation.gif" alt="drawing" width="300"/>
<img src="https://raw.githubusercontent.com/bradygm/PID-controller-simulator/main/media/squareWave2.png" alt="drawing" width="360"/>

Obviously the arm can't teleport from one side to the other, but do your best to get your arm to the goal positions as quickly as possible. Go to the right, then stop the arm. Go to the left, and stop the arm. 

This is pretty tricky, but give it a couple attempts.

In order to more quickly see your results from the simulation, you can choose to **temporarily turn off the animations**. You can always turn it back on whenever to see your final results. Or leave it on all the time if you have great patience 😉.

In [42]:
# Creates the CustomInput object for our input torque sequence
custom_torques_square = CustomInput(-1,1)
# Starts recording a new torque sequence
custom_torques_square.record()

In [43]:
#@markdown How long you want the simulation to run for
end_time = 10.0 #@param{type:"slider", min:5.0, max:30, step:1} 
period = 10.0
# The starting angle of the arm
start_angle = 0.0 
#@markdown Toggle animation of figures on or off. Turning all off will result in quick results.
animate_plots = False #@param {type:"boolean"}
animate_sim = True #@param{type:"boolean"}

dt = 0.05 # the length of the time step
time_list = np.arange(0, end_time, dt)
list_len = len(time_list)

# Initialize the RobotArm and save as my_robot_arm
my_robot_arm = RobotArm(start_angle, dt)

# Takes our custom torque input and interpolates it for all time steps
torques_interp = interp_values(custom_torques_square.values, end_time, dt)
ang_hist = []
goal_ang_interp = signal.square(2 * np.pi * time_list/period)

# Loop over the time steps
for i in range(list_len):
    my_robot_arm.step_forward(torques_interp[i])
    ang_hist.append(my_robot_arm.get_ang_position())

display_results(goal_ang_interp)

10.219490051269531 sec


## PID Controller Magic

That was hard, right? Now that we have tried a couple times to get the arm to follow the goal path, we are now going to write a controller to have it do the hard work for us!



### P Controller

First, let's start with creating a proportional controller (The "P" part of PID). We will create a function called `p_controller` that takes in variables `P`, `goal_position`, and `current_position`. What we want to do is determine what the control input should be using those three variables. The first step is to find the difference between where we want the arm to be, the goal position, and where it is now, the current position. This is called the error. 

Using the error, we then can calculate what our torque `u` should be. We do this by multiplying the error by the value `P` to determine how strongly the arm should react to a given error. 

**Fill in the rest of the equation** where you see `u=` where it sets the value of the `u` to whatever you put on the right side of the equation. This `u` is what the torque will be and it is returned as the output of this function. 

In [44]:
def p_controller(P, goal_position, current_position):
    # We first calculate the error
    error = goal_position-current_position
    # ADD YOUR CODE HERE
    u = error*P 
    return u

Let's test out your equation on the sim! If you need help or want to double check your answer, double click on this cell to see an example solution.

<!-- 

u = P*error

-->

The code block below is very similar to the one before but it now has two additional parameters. It has the `P` variable which we pass into our new function `p_controller`. Give it a try with the default values first to see the results. Note that we have turned off the animations by default for quicker results. 

In [45]:
#@markdown How long you want the simulation to run for
end_time = 30 #@param{type:"slider", min:5.0, max:30, step:1}
#@markdown The length of each cycle of our square wave goal position
period = 60 #@param{type:"slider", min:1, max: 60.0, step:1}
# The starting angle of the arm
start_angle = 0.0 
#@markdown Toggle animation of figures on or off. Turning all off will result in quick results.
animate_plots = False #@param{type:"boolean"}
animate_sim = False #@param{type:"boolean"}

dt = 0.05 # the length of the time step
time_list = np.arange(0, end_time, dt)
list_len = len(time_list)

#@markdown Proportional Gain
P = 0.38 #@param{type:"slider", min:0.01, max:5.0, step:0.01}

# Initialize the RobotArm and save as my_robot_arm
my_robot_arm = RobotArm(start_angle, dt)

torques_interp = []
ang_hist = []
goal_ang_interp = signal.square(2 * np.pi * time_list/period)

for i in range(list_len):
    torque = p_controller(P, goal_ang_interp[i], my_robot_arm.get_ang_position())
    torques_interp.append(torque) # save torque so we can draw it
    my_robot_arm.step_forward(torque)
    ang_hist.append(my_robot_arm.get_ang_position()) # save angle so we can draw it

display_results(goal_ang_interp)

0.40658092498779297 sec


How did it do? Try a couple different values for P so see how close you can get the arm to follow the goal. 

The other parameter we added to the code block above is called `period` which allows you to adjust the length of each cycle of the square wave which we use as the goal angle of the arm. Try increasing the end time and the period length in order to see longer time behaviors of the system.

One thing you can try is maxing out the end time, period, and the P value. What happens? You may see that the oscillations actually start to get bigger and bigger as time goes on. This is an **unstable** system. 

Now try with a lower `P` value, something like 0.05. You will see that there are less oscillations, but it gets to the goal angle much too slow, especially to follow the original square wave with a period of 10. 


**Should I include the images, or let them try? Let them, but at some point would be nice to just show so they don't get bored of trying each time? Test. Late add in images and say should look like below**

### PD Controller

How can we improve upon the proportional controller? We want to find the balance of moving quickly to the goal angle, but not move to quickly which leads to overshooting. 

PD controller to the rescue!

We can add in another part to our controller the keeps our arm from moving to quickly, or having too large of a velocity. We do this by multiplying our velocity by a scaling value `D` and subtracting it from our control input `u`.

We are now ready to create our new function 25332543 also have current celocity and D

In [46]:
def pd_controller(P, D, goal_position, current_position, current_velocity):
    # We first calculate the error
    error = goal_position-current_position
    # ADD YOUR CODE HERE
    u = error*P - current_velocity*D
    return u

Again, if you need help or want to double check your answer, double click on this cell to see an example solution.

<!-- 

u = P*error - D*current_velocity

-->

We've now added the new variable `D` which is passed into our function `pd_controller`. Give it a try with the default values first to see the results. 

In [75]:
#@markdown How long you want the simulation to run for
end_time = 10 #@param{type:"slider", min:5.0, max:30, step:1}
#@markdown The length of each cycle of our square wave goal position
period = 10 #@param{type:"slider", min:1, max: 60.0, step:1}
# The starting angle of the arm
start_angle = 0.0 
#@markdown Toggle animation of figures on or off. Turning all off will result in quick results.
animate_plots = False #@param{type:"boolean"}
animate_sim = False #@param{type:"boolean"}

dt = 0.05 # the length of the time step
time_list = np.arange(0, end_time, dt)
list_len = len(time_list)

#@markdown Proportional Gain
P = 0.19 #@param{type:"slider", min:0.01, max:5.0, step:0.01}
D = 0 #@param{type:"slider", min:0.0, max:5.0, step:0.01}

# Initialize the RobotArm and save as my_robot_arm
my_robot_arm = RobotArm(start_angle, dt)

torques_interp = []
ang_hist = []
goal_ang_interp = signal.square(2 * np.pi * time_list/period)

for i in range(list_len):
    torque = pd_controller(P, D, 
                           goal_ang_interp[i], 
                           my_robot_arm.get_ang_position(),
                           my_robot_arm.get_ang_velocity())
    torques_interp.append(torque) # save torque so we can draw it
    my_robot_arm.step_forward(torque)
    ang_hist.append(my_robot_arm.get_ang_position()) # save angle so we can draw it

submit_response(Name="AnonUser")
display_results(goal_ang_interp)


Total Error: 8.578432259142595
0.7112979888916016 sec


You will find that you can start cranking up `P` if you also use a high value of `D` to offset it. 

What happens if you crank `D` all the way up? If it is too high, it ca. 

Leaderboard: https://docs.google.com/spreadsheets/d/1pbKPOv4ZjdaPNsYJWK1wwlBWOSiPoD-gEATfgp6heQs/edit?usp=sharing Edit name. Turn off by default?

You should be able to get something that get to the line pretty quickly in under ffj seconds. And could look like this?

### Test your final controller

Custom goal angle sequence. Now rather than moving the slider to input the torque, we are creating a sequence for the goal angle of the arm that your controller will try to follow. 



In [ ]:
# Creates the CustomInput object for our input goal angle sequence
custom_goal_angle = CustomInput(-3.14,3.14)
# Starts recording a new goal angle sequence
custom_goal_angle.record()

Now let's see how it does!

Copy over your `P` and `D` values you found that worked best in .f .f .d

In [ ]:
#@markdown How long you want the simulation to run for
end_time = 10 #@param{type:"slider", min:5.0, max:30, step:1}
#@markdown The length of each cycle of our square wave goal position
period = 10 #@param{type:"slider", min:1, max: 60.0, step:1}
# The starting angle of the arm
start_angle = 0.0 
#@markdown Toggle animation of figures on or off. Turning all off will result in quick results.
animate_plots = True #@param{type:"boolean"}
animate_sim = True #@param{type:"boolean"}

dt = 0.05 # the length of the time step
time_list = np.arange(0, end_time, dt)
list_len = len(time_list)

#@markdown Proportional Gain
P = 5 #@param{type:"slider", min:0.01, max:5.0, step:0.01}
D = 2.11 #@param{type:"slider", min:0.0, max:5.0, step:0.01}

# Initialize the RobotArm and save as my_robot_arm
my_robot_arm = RobotArm(start_angle, dt)

torques_interp = []
ang_hist = []
goal_ang_interp = interp_values(custom_goal_angle.values, end_time, dt)

for i in range(list_len):
    torque = pd_controller(P, D, 
                           goal_ang_interp[i], 
                           my_robot_arm.get_ang_position(),
                           my_robot_arm.get_ang_velocity())
    torques_interp.append(torque) # save torque so we can draw it
    my_robot_arm.step_forward(torque)
    ang_hist.append(my_robot_arm.get_ang_position()) # save angle so we can draw it

display_results(goal_ang_interp)

20.418351650238037 sec


## Final sim thing to mess around and try things



#### Controller


Probably split up the controller into different ones. Start with p, then pd. Then we do pid? Or later. Maybe...

In [ ]:
class PID_Controller:

    def __init__(self, k_p=1, k_i=0, k_d=0):
        self.control_limit = 0 #not currently used
        self.error_sum = 0

        self.k_p = k_p
        self.k_i = k_i
        self.k_d = k_d

    def p_control(self, desired_position, current_position):
        error = desired_position-current_position
        return self.k_p*error

    def pd_control(self, desired_position, current_position, current_velocity):
        error = desired_position-current_position
        return self.k_p*error - self.k_d*current_velocity

    def pid_control(self, desired_position, current_position, current_velocity):
        error = desired_position-current_position
        self.error_sum += error
        return self.k_p*error + self.k_i*self.error_sum - self.k_d*current_velocity

In [ ]:
end_time = 10.0 #@param{type:"slider", min:5.0, max:30, step:1}
start_angle = 0.1 #@param{type:"slider", min:-3.14, max:3.14, step:0.1}
k_p = 6.71 #@param{type:"slider", min:0.0, max:20.0, step:0.01}
k_i = 0.0 #@param{type:"slider", min:0.0, max:5.0, step:0.01}
k_d = 3.34 #@param{type:"slider", min:0.0, max:5.0, step:0.01}
dt = 0.05 #@param
animate_plots = False #@param{type:"boolean"}
animate_sim = False #@param{type:"boolean"}
time_list = np.arange(0, end_time, dt)
list_len = len(time_list)

# Initialize the RobotArm and save as my_robot_arm
my_robot_arm = RobotArm(start_angle, dt)
# Initialize our controller
controller = PID_Controller(k_p, k_i, k_d)

# torques_interp = interp_values(custom_torques.values, end_time, dt)
torques_interp = []
goal_ang_interp = signal.square(.1*2 * np.pi * time_list)
# goal_ang_interp = interp_values(custom_goal_angle.values, end_time, dt)


ang_hist = []

for i in range(list_len):
    # my_robot_arm.step_forward(torques_interp[i])
    # my_robot_arm.step_forward(my_robot_arm.p_control(.5))
    torque = controller.pd_control(goal_ang_interp[i], my_robot_arm.get_ang_position(), my_robot_arm.get_ang_velocity())
    torques_interp.append(torque)
    my_robot_arm.step_forward(torque)
    # cur_torque = controller.pid_control(0, my_robot_arm.get_ang_position(), my_robot_arm.get_ang_velocity())
    # my_robot_arm.step_forward(cur_torque)
    # my_robot_arm.step_forward(0)
    ang_hist.append(my_robot_arm.get_ang_position())

display_results(goal_ang_interp)

0.4173905849456787 sec


Try turning gravity on or off. Start at angle and see what it does. 

Remember can turn off animation for quicker. Reminder. 

Calculate error. Can sum to give them a metric. But also put a limit? Make sure ti would lead to something interesting. 

Look at the graph of troque. Would have been hard for us to come up with, right? Math is great. 

Double click on this cell to view a working answer if you can't get it to work.

<!-- 

Write your comments here 

-->

<hr>

Then they mess around with the slider being the torque. We can then say how rather than a slider with us puting in the torque, we use a PID controller.

Allow for changing the trajectory. Also a slider? How easily change? Swap out or comment out lines?

Now at the end of example 1 things you could try. 
* Change arm parameters and see how it does with the pid parameters you chose. How does it change?
* Change the start config.
* Add gravity. to image veritical Gravity things to think about and try. Why didn't work? Link to info on how to modify if want. 
* add limit (or have in line follower)
* amplitude of wave

# Example 2: Line following robot

Could start with or maybe only have a simple straight line. Car going from left to right. They modify the starting position laterally (starting crosstrack error). Maybe move onto a more interesting track or we have a set of tracks. Would be really cool if they could draw their own paths. 

Ideas
* Could have live plot showing cross track error (can be used for explaining the D or I of PID but also just fun to see)
* Have a benchmark set for them to get a score and maybe have leaderboard optional
* Fun parameters we could expose with sliders or text entry or could just be clearly marked in the code
  * Vehicle speed
  * Max turn rate/angle
  * Look ahead distance when calculating cross track error?

# Notes and thoughts

* How make the quality of our presentation really good? 
  * Zoom clear background thing?
  * OBS?
  * Look at other examples?
* Test on people
    * Test on people without any instruction to test the written instructions. 
    * Test with me teaching to test pace and length

# Before final save
* Open table of contents
* hide beginning cells
* clear outputs

# Indu

3rd Graph: Control input torques_interp - done

On animated plot: Add goal trajectory ghosted

Feel free to clean display_results()

Any speed ups? But probably okay. 


In [32]:



submit_response(Name="AnonUser", Error=999999999999) 



<Response [200]>